In [1]:
import pandas as pd 
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
import matplotlib.pyplot as plt

In [2]:
dados ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/Conv1d/Dominio da Frequencia/Todos os arrays/dados_freq.npy'
rotulos ='C:/Users/matto/Desktop/Leandro/Mestrado PPCIC/Dissertação/novo tema/Novo cj de dados/rotulos.npy'

In [3]:
import numpy as np
X = np.load(dados)
y = np.load(rotulos)

In [4]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [5]:
X_train.shape

(1493, 200, 1)

In [6]:
X_train.tolist()
y_train.tolist()
X_test.tolist()
y_test.tolist()



[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,


In [7]:
batch = int(len(y_train)/30)
batch

49

In [8]:
from keras.utils import np_utils
y_train_convertido = np_utils.to_categorical(y_train)
y_test_convertido =  np_utils.to_categorical(y_test)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout

In [10]:
import optuna
from keras.optimizers import SGD

def objective(trial):
    
    # Definindo o espaço de busca dos hiperparâmetros
    
    num_layers = trial.suggest_int('num_layers', 1, 5)
    dense_neurons = trial.suggest_int('dense_neurons', 20, 4000, log=True)
    dropout = trial.suggest_int('dropout', 2, 5, step=1) / 10.0
    learning_rate = trial.suggest_categorical('learning_rate', [0.001, 0.003,0.005,0.007,0.01,0.03,0.05, 0.07])
    
    modelo = Sequential()
    
    for i in range(num_layers):
        if i == 0:
            modelo.add(Dense(dense_neurons, input_dim=200, kernel_initializer='normal',activation='relu'))
            modelo.add(Dropout(dropout))
        else:
            modelo.add(Dense(dense_neurons, kernel_initializer='normal',activation='relu'))
            modelo.add(Dropout(dropout))

    modelo.add(Dense(2, kernel_initializer='normal',activation = 'softmax'))
                     
    otimizador = SGD(learning_rate=learning_rate) 
    modelo.compile(loss="categorical_crossentropy",optimizer=otimizador,metrics =["acc"])
    historico = modelo.fit(X_train,y_train_convertido, epochs= 300, batch_size = batch, validation_data=(X_test,y_test_convertido),verbose = 1)
    
    return historico.history['val_acc'][-1]

In [11]:
import csv

# criação do objeto de estudo
study = optuna.create_study(direction="maximize")

# abertura do arquivo CSV em modo de escrita
with open("results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["trial", "accuracy", "hyperparameters"])  # cabeçalho

    # iteração dos trials
    
    study.optimize(objective, n_trials=80)
    best_trial = study.best_trial
    i = study.trials[0].number  # Obtém o número do primeiro trial realizado
    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

    # escrita dos valores no arquivo CSV
    writer.writerow([i, best_trial.value, best_trial.params])

    print("Trial {}: Accuracy = {}".format(i, best_trial.value))


print(study.best_params)



[I 2023-07-09 21:55:00,300] A new study created in memory with name: no-name-3e5c948d-4054-4965-9b7b-3105c7f5dd36


Epoch 1/300
31/31 [==============================] - 1s 14ms/step - loss: 0.6218 - acc: 0.8399 - val_loss: 0.5595 - val_acc: 0.8752
Epoch 2/300
31/31 [==============================] - 0s 4ms/step - loss: 0.5464 - acc: 0.8607 - val_loss: 0.4910 - val_acc: 0.8752
Epoch 3/300
31/31 [==============================] - 0s 4ms/step - loss: 0.5121 - acc: 0.8607 - val_loss: 0.4577 - val_acc: 0.8752
Epoch 4/300
31/31 [==============================] - 0s 4ms/step - loss: 0.4958 - acc: 0.8607 - val_loss: 0.4452 - val_acc: 0.8752
Epoch 5/300
31/31 [==============================] - 0s 5ms/step - loss: 0.4794 - acc: 0.8607 - val_loss: 0.4311 - val_acc: 0.8752
Epoch 6/300
31/31 [==============================] - 0s 5ms/step - loss: 0.4702 - acc: 0.8607 - val_loss: 0.4218 - val_acc: 0.8752
Epoch 7/300
31/31 [==============================] - 0s 6ms/step - loss: 0.4580 - acc: 0.8607 - val_loss: 0.4096 - val_acc: 0.8752
Epoch 8/300
31/31 [==============================] - 0s 6ms/step - loss: 0.4422 - 

31/31 [==============================] - 0s 5ms/step - loss: 0.1343 - acc: 0.9645 - val_loss: 0.1047 - val_acc: 0.9766
Epoch 64/300
31/31 [==============================] - 0s 5ms/step - loss: 0.1237 - acc: 0.9618 - val_loss: 0.1064 - val_acc: 0.9766
Epoch 65/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1188 - acc: 0.9678 - val_loss: 0.1007 - val_acc: 0.9797
Epoch 66/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1152 - acc: 0.9699 - val_loss: 0.0997 - val_acc: 0.9797
Epoch 67/300
31/31 [==============================] - 0s 5ms/step - loss: 0.1086 - acc: 0.9745 - val_loss: 0.1002 - val_acc: 0.9782
Epoch 68/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1173 - acc: 0.9685 - val_loss: 0.0986 - val_acc: 0.9797
Epoch 69/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1164 - acc: 0.9672 - val_loss: 0.0971 - val_acc: 0.9813
Epoch 70/300
31/31 [==============================] - 0s 6ms/step - loss: 0.1132 - acc: 0

Epoch 125/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0666 - acc: 0.9779 - val_loss: 0.0640 - val_acc: 0.9813
Epoch 126/300
31/31 [==============================] - 0s 5ms/step - loss: 0.0633 - acc: 0.9799 - val_loss: 0.0674 - val_acc: 0.9797
Epoch 127/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0660 - acc: 0.9792 - val_loss: 0.0729 - val_acc: 0.9797
Epoch 128/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0596 - acc: 0.9786 - val_loss: 0.0654 - val_acc: 0.9813
Epoch 129/300
31/31 [==============================] - 0s 5ms/step - loss: 0.0557 - acc: 0.9812 - val_loss: 0.0621 - val_acc: 0.9813
Epoch 130/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0571 - acc: 0.9833 - val_loss: 0.0672 - val_acc: 0.9813
Epoch 131/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0664 - acc: 0.9759 - val_loss: 0.0673 - val_acc: 0.9813
Epoch 132/300
31/31 [==============================] - 0s 6ms/step - 

31/31 [==============================] - 0s 4ms/step - loss: 0.0553 - acc: 0.9812 - val_loss: 0.0672 - val_acc: 0.9782
Epoch 187/300
31/31 [==============================] - 0s 5ms/step - loss: 0.0442 - acc: 0.9853 - val_loss: 0.0660 - val_acc: 0.9782
Epoch 188/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0471 - acc: 0.9846 - val_loss: 0.0626 - val_acc: 0.9797
Epoch 189/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0461 - acc: 0.9873 - val_loss: 0.0593 - val_acc: 0.9797
Epoch 190/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0443 - acc: 0.9853 - val_loss: 0.0604 - val_acc: 0.9813
Epoch 191/300
31/31 [==============================] - 0s 5ms/step - loss: 0.0412 - acc: 0.9826 - val_loss: 0.0646 - val_acc: 0.9813
Epoch 192/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0408 - acc: 0.9853 - val_loss: 0.0691 - val_acc: 0.9813
Epoch 193/300
31/31 [==============================] - 0s 6ms/step - loss: 0.0614 -

...

31/31 [==============================] - 0s 10ms/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.0760 - val_acc: 0.9813
Epoch 125/300
31/31 [==============================] - 0s 15ms/step - loss: 0.0025 - acc: 0.9993 - val_loss: 0.0602 - val_acc: 0.9860
Epoch 126/300
31/31 [==============================] - 0s 10ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0752 - val_acc: 0.9813
Epoch 127/300
31/31 [==============================] - 0s 13ms/step - loss: 0.0062 - acc: 0.9980 - val_loss: 0.0590 - val_acc: 0.9828
Epoch 128/300
31/31 [==============================] - 0s 9ms/step - loss: 0.0022 - acc: 0.9993 - val_loss: 0.0579 - val_acc: 0.9860
Epoch 129/300
31/31 [==============================] - 0s 9ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0638 - val_acc: 0.9813
Epoch 130/300
31/31 [==============================] - 0s 11ms/step - loss: 0.0139 - acc: 0.9940 - val_loss: 0.0554 - val_acc: 0.9860
Epoch 131/300
31/31 [==============================] - 0s 15ms/step - loss: 0.

31/31 [==============================] - 0s 9ms/step - loss: 0.0013 - acc: 0.9993 - val_loss: 0.0941 - val_acc: 0.9797
Epoch 186/300
31/31 [==============================] - 0s 8ms/step - loss: 9.2485e-04 - acc: 1.0000 - val_loss: 0.0902 - val_acc: 0.9797
Epoch 187/300
31/31 [==============================] - 0s 9ms/step - loss: 5.8551e-04 - acc: 1.0000 - val_loss: 0.1015 - val_acc: 0.9797
Epoch 188/300
31/31 [==============================] - 1s 17ms/step - loss: 8.6100e-04 - acc: 1.0000 - val_loss: 0.0909 - val_acc: 0.9813
Epoch 189/300
31/31 [==============================] - 0s 10ms/step - loss: 5.2847e-04 - acc: 1.0000 - val_loss: 0.0891 - val_acc: 0.9813
Epoch 190/300
31/31 [==============================] - 0s 9ms/step - loss: 0.0021 - acc: 0.9993 - val_loss: 0.0854 - val_acc: 0.9813
Epoch 191/300
31/31 [==============================] - 0s 9ms/step - loss: 0.0019 - acc: 0.9993 - val_loss: 0.0909 - val_acc: 0.9828
Epoch 192/300
31/31 [==============================] - 0s 9ms/ste

31/31 [==============================] - 0s 13ms/step - loss: 0.0014 - acc: 0.9993 - val_loss: 0.0893 - val_acc: 0.9813
Epoch 246/300
31/31 [==============================] - 0s 12ms/step - loss: 5.6747e-04 - acc: 1.0000 - val_loss: 0.0892 - val_acc: 0.9828
Epoch 247/300
31/31 [==============================] - 0s 8ms/step - loss: 3.5590e-04 - acc: 1.0000 - val_loss: 0.0865 - val_acc: 0.9844
Epoch 248/300
31/31 [==============================] - 0s 9ms/step - loss: 7.2768e-04 - acc: 1.0000 - val_loss: 0.0895 - val_acc: 0.9828
Epoch 249/300
31/31 [==============================] - 0s 10ms/step - loss: 8.2031e-04 - acc: 0.9993 - val_loss: 0.0978 - val_acc: 0.9782
Epoch 250/300
31/31 [==============================] - 0s 11ms/step - loss: 1.9730e-04 - acc: 1.0000 - val_loss: 0.0971 - val_acc: 0.9782
Epoch 251/300
31/31 [==============================] - 0s 13ms/step - loss: 2.1269e-04 - acc: 1.0000 - val_loss: 0.0925 - val_acc: 0.9813
Epoch 252/300
31/31 [==============================] -

[I 2023-07-10 06:47:03,307] Trial 79 finished with value: 0.9781591296195984 and parameters: {'num_layers': 5, 'dense_neurons': 215, 'dropout': 2, 'learning_rate': 0.05}. Best is trial 25 with value: 0.9859594106674194.


Trial 0: Accuracy = 0.9859594106674194
Trial 0: Accuracy = 0.9859594106674194
{'num_layers': 5, 'dense_neurons': 215, 'dropout': 4, 'learning_rate': 0.03}
